In [28]:
import ROOT as r
import math
import array
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from math import pi

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, '../include')
sys.path.insert(0, include_dir)
#__________________________________________________
import ERROR
import CONFIG
import DBPARSE
from ASYMMETRY import *
from UTILITIES import *
from SIMFITS import DistributionFits
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend
import CORRECTIONS
import CALCULATE
from joblib import Parallel, delayed
import importlib
#importlib.reload(ASYMMETRY)

In [29]:
pas="1"
config="3"

In [30]:
muN=1.9103 #minus

In [31]:
import sys
import re

class Filter:
    def __init__(self):
        self.old_stderr = sys.stderr
        sys.stderr = self

    def write(self, message):
        # Change the pattern here to match the specific ROOT errors
        if not re.search(r"Error in <TList::Clear>:", message):
            self.old_stderr.write(message)

    def flush(self):
        pass

# Apply the filter
filter = Filter()



# Set Config

In [32]:
#in GeV^2
m=.939565
muN=-1.9103
if config=="2":
    Q2=3
    tau=Q2/(4*m**2)
    theta=29.5*pi/180
if config=="3":
    Q2=6.83
    #Q2=6.62 #seans number
    tau=Q2/(4*m**2)
    theta=36.5*pi/180
if config=="4":
    Q2=9.82
    tau=Q2/(4*m**2)
    theta=35*pi/180

In [33]:
W2min=CONFIG.Function_JSON("W2min",f"../config/cuts{config}.cfg")
W2max=CONFIG.Function_JSON("W2max",f"../config/cuts{config}.cfg")
dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
dymin=CONFIG.Function_JSON("dymin",f"../config/cuts{config}.cfg")
dymax=CONFIG.Function_JSON("dymax",f"../config/cuts{config}.cfg")
dybgmin=CONFIG.Function_JSON("dybgmin",f"../config/cuts{config}.cfg")
dybgmax=CONFIG.Function_JSON("dybgmax",f"../config/cuts{config}.cfg")
coinmin=CONFIG.Function_JSON("coinmin",f"../config/cuts{config}.cfg")
coinmax=CONFIG.Function_JSON("coinmax",f"../config/cuts{config}.cfg")
Q2=CONFIG.Function_JSON("Q2",f"../config/cuts{config}.cfg")
print(f'W2min: {W2min}')
print(f'W2max: {W2max}')
print(f'dxmin: {dxmin}')
print(f'dxmax: {dxmax}')
print(f'dymin: {dymin}')
print(f'dymax: {dymax}')
print(f'dybgmin: {dybgmin}')
print(f'dybgmax: {dybgmax}')
print(f'Q2: {Q2}')
print(f'Coin Min: {coinmin} Coin Max: {coinmax}')

W2min: 0
W2max: 1.2
dxmin: -0.4
dxmax: 0.402
dymin: -0.9895
dymax: 0.9895
dybgmin: -1.2
dybgmax: 1.2
Q2: 6.83
Coin Min: 82 Coin Max: 106


In [34]:
cut="w2"
value=[W2min,W2max]


In [35]:
def Function_APHYSSENS(config,cut,value):
    import ROOT as r
    import math
    import array
    import os
    import sys
    import matplotlib.pyplot as plt
    import numpy as np

    #______________Add include directory_______________
    current_dir = os.getcwd()
    include_dir = os.path.join(current_dir, '../include')
    sys.path.insert(0, include_dir)
    #__________________________________________________
    import ERROR
    import CONFIG
    import DBPARSE
    import ASYMMETRY 
    import UTILITIES 
    from SIMFITS import DistributionFits
    from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend
    import CALCULATE
    import CORRECTIONS
    from joblib import Parallel, delayed
    
    dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
    dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
    lower_bound=dxmin
    upper_bound=dxmax
    Raw=ASYMMETRY.Function_ASYMMETRYSENS(config,cut,value)
    print("Raw Asymmetry Calculated")
    
    Fit=ASYMMETRY.Function_FITDXSENS(config,cut,value)
    print("dx Histogram Fit Completed")
    
    data,bg,totalFit,simp,simn=Fit
    bin_centers,bin_contents=bg
    numBG=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
    bin_centers,bin_contents=data
    numTotal=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
    bin_centers,bin_contents=simp
    numP=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
    
    accResult,bgResult,fp=CORRECTIONS.Function_ANALYZEROOTFILE(config,bg,simp,totalFit)
    
    bin_centers,bin_contents=simn
    Y=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
    TrueYield=Raw[3]
    print("f A combinations Determined")
    
    Phys=ASYMMETRY.Function_APHYS(config,pas,Raw,accResult,bgResult,fp)
    print("Aphys Calculated")
    
    weighted_A,weighted_A_E,fbackg,rawA,rawAE=Phys
    print('Returned: Aphys, Aphys_E, fbg, cut, cutvalue')
    R,RE=CALCULATE.Function_CALCGEN(config,weighted_A,weighted_A_E)
    return weighted_A,weighted_A_E,fbackg,cut,value,Y,R,RE,rawA,rawAE,np.mean(Raw[1])

In [36]:
Results=Function_APHYSSENS(config,cut,value)

Config: 3
Raw Asymmetry Calculated
GEN3
dx Histogram Fit Completed
f A combinations Determined
fA Error:0.002503162538873578+-0.0009666109542987236
f Error:0.5866+-0.006453923361466381
Aphys Calculated
Returned: Aphys, Aphys_E, fbg, cut, cutvalue


In [37]:
A=Results[0]
AE=Results[1]

R=abs(Results[6])
RE=Results[7]

print(f"Kin{config} - muN GEN/GMN: {abs(R*muN)}+/-{abs(RE*muN)}")

np.save(f'SavedArrays/GEN{config}/Pass{pas}/GEGM.npy',R)
np.save(f'SavedArrays/GEN{config}/Pass{pas}/GEGME.npy',RE)

Kin3 - muN GEN/GMN: 0.5617247598820028+/-0.3766043098397711


In [38]:
A

-0.14193687734413604